![Demo architecture](img/QPSK_system_block_diagrams_Tx_only.svg)

## Initialisation

Let's include pynq libraries and our own drivers

In [ ]:
from pynq import Overlay
from qpsk_tx import QPSK_TX_Core, QPSK_TX
from qpsk_rx import QPSK_RX_Core, QPSK_RX
from rfsoc_plots import *
from dma_gen import DmaGen
from xrfclk import xrfclk_set_freq
from xrfdc import *

Set up LMX clock synthesizer to 204.8 MHz

In [ ]:
xrfclk_set_freq(204.8)

Now we're ready to load our bitstream to the PL

In [ ]:
ol = Overlay("rfsoc_pynq_demo_v5.1.5.bit")

### Configuring the RF Data Converters

We are going to use one of the DAC blocks to transmit our signal.
We'll need to generate a clock using one of the on-board synthesizers, tell the data converters about it, and set an NCO frequency --- controlling our signal's carrier frequency. 

![Diagram of on-board synth, DAC block with PLL, and NCO](img/RF_DAC.svg)

Make our RF data converter object

In [ ]:
dc = ol.usp_rf_data_converter_0
dac_tile  = dc.dac_tiles[1]
dac_block = dac_tile.blocks[2]

Set DAC's clock source and NCO/mixer frequency

In [ ]:
dac_tile.ShutDown()
dac_tile.DynamicPLLConfig(XRF_CLK_SRC_PLL,204.8,204.8)

dac_mixer = {"Freq": 84,
             "CoarseMixFreq": XRF_DC_COARSE_MIX_SAMPLE_FREQ_BY_TWO,
             "CoarseMixMode": XRF_DC_COARSE_MIX_MODE_C2C_C2R,
             "FineMixerMode": XRF_DC_FINE_MIXER_MOD_COMPLX_TO_REAL}
dac_block.MixerSettings = dac_mixer
dac_tile.SetupFIFO(True)
dac_tile.StartUp()

See `xrfdc.py` for implementation, showing how easily we can wrap existing C drivers as object oriented python drivers!

### Initialise our QPSK TX Design

Finally, we make an instance of a driver for the QPSK transmit logic. This controls the signal processing performed on the signal, as well as tapping off the data path, allowing us to interactively plot the intermediate signals.

See `qpsk_tx.py` for the implementation of the driver, exposing registers as object properties using some data-driven generation

In [ ]:
tx = ol.qpsk_tx

## Visualising internal signals

Now that the transmitter is configured and constantly running in the background, we can capture intermediate signals and visualise them interactively.

Our raw, binary data is carried by two parts ("I" and "Q") in a complex signal.  These each have a value of 1 or -1, conveying 1 bit of information.

Let's make a time domain plot of this signal --- straight after our `symbol generation` block.

### Raw QPSK symbols

Here we generate a time domain plot that updates in real-time. We make use of Pynq's DMA drivers, and a python Timer for scheduled transfers.

In [ ]:
cplot = IQTimePlot(tx.getManySymbols(N=10), 500, resampling_fun=resample_pick)
#cplot.set_line_mode(markers=True, lines=True)

dg = DmaGen(cplot.add_data, tx.getSymbols, 0.05)
ipw.VBox([cplot.get_widget(), 
          ipw.HBox(GraphControls(dg))])

We can stream in more live data using the play/stop buttons.

Alright, cool! But changing this signal instantaneously from -1 to 1 means there's a near infinite bandwidth (i.e. has components in all frequencies!). To help suppress this for successful transmission, we perform some `pulse shaping`.

### Pulse shaping

Next we make the same plot, but with data from after our pulse shaping unit. First we'll see this in the time domain, then we will look at the same signal in the frequency domain.

In [ ]:
iq_plot = IQTimePlot(tx.getManyShapedTime(N=10), 4000,w=800)
iq_dg = DmaGen(iq_plot.add_data, tx.getShapedTime, 0.05)

fa_plot = HWFreqPlot(tx.getShapedFFT(), 4000, avg_n=4,w=800)
fa_dg = DmaGen(fa_plot.add_frame, tx.getShapedFFT, 0.3)

tab1 = ipw.Tab([ipw.VBox([iq_plot.get_widget(), ipw.HBox(GraphControls(iq_dg))]),
                ipw.VBox([fa_plot.get_widget(), ipw.HBox(GraphControls(fa_dg))])
               ])
tab1.set_title(0, 'Time domain')
tab1.set_title(1, 'Frequency domain')
tab1

## Dynamic Control

We can also use widgets to control our PL settings visually. For example, our mixer's NCO frequency and the gain of the signal.

First, let's write some JSON that describes our settings and their valid ranges.

In [ ]:
from configuration import Configuration
import json

schema_json = """
{
  "title": "TX Settings",
  "type": "object",
  "properties": {
    "Freq": {
      "description": "NCO Frequency (MHz)",
      "type": "number",
      "minimum": 0.0,
      "maximum": 100.0,
      "default": 84.0
    },
    "Gain": {
      "description": "Relative Gain",
      "type": "number",
      "minimum": 0.0,
      "maximum": 1.0,
      "default": 1
    }
  }
}
"""


Now, let's define a function that will take a dictionary of these settings and apply them to our system. We must ensure that only one instance of this is run at a given time (using Python's `Lock()`) because our driver isn't thread safe (yet).

In [ ]:
from threading import Thread, Lock

tx_mutex = Lock()

def update_tx_settings(config):
    # Acquire lock for mutual exclusion on RF driver
    if tx_mutex.acquire(blocking=False):
        # Now work in a try block so we will always release the lock, even if our code fails
        try:
            if 'Gain' in config.keys():
                tx.axi_qpsk_tx.output_gain = int(config['Gain']*(2**32-1))
            if 'Freq' in config.keys():
                dac_tile.ShutDown()
                mixer_cfg = dac_block.MixerSettings
                mixer_cfg['Freq'] = config['Freq']
                dac_block.MixerSettings = mixer_cfg
                dac_tile.StartUp()
        except:
            pass
        # Remember to release the lock
        tx_mutex.release()

Finally we can make our widget and play with the values.

See the effect of changing the gain in the FFT and time domain signal. The effect of the NCO mixer is a bit more subtle --- keep an eye out for a look of suppressed horror on Kenny's face as his receiver gets out of sync!

In [ ]:
cfg = Configuration(json.loads(schema_json))
gui = cfg.interact(callback=update_tx_settings)
display(gui)

# QPSK Receiver

Now that the transmitter is configured, we start to play with the receiver side.

![Demo architecture](img/QPSK_system_block_diagrams_Rx_only.svg)

## Config

First, we configure the RF ADC block. The ADC tile's PLL is configured, and make sure to match the block's mixer frequency with the transmitting side! For us that's 84 MHz unless you've been playing with the sliders above...

In [ ]:
adc_tile  = dc.adc_tiles[0]
adc_block = adc_tile.blocks[0]

In [ ]:
adc_tile.ShutDown()
adc_tile.DynamicPLLConfig(XRF_CLK_SRC_PLL,204.8,1024)

adc_mixer = {"Freq": 84.000,
             "CoarseMixFreq": XRF_DC_COARSE_MIX_SAMPLE_FREQ_BY_TWO,
             "CoarseMixMode": 2,
             "FineMixerMode": 3,
             "EventSource": 2}

adc_block.MixerSettings = adc_mixer
adc_tile.SetupFIFO(True)
adc_tile.StartUp()

In [ ]:
rx = ol.qpsk_rx

## Decimated signal

Now we can start inspecting some of the received signals.
Below, we can grab data from multiple parts of our RX signal path and plot them in the time domain, frequency domain, or as a constellation.

Feel free to change the first line to swap between inspecting the raw signal after decimation, after coarse synchronisation, or after the RRC.

In [ ]:
taps = {
    'Decimated': 
      {'get': rx.getDecimated,    'getMany': rx.getManyDecimated,    'fs':1024},
    'CoarseSynced': 
      {'get': rx.getCoarseSynced, 'getMany': rx.getManyCoarseSynced, 'fs':4096},
    'RRCed': 
      {'get': rx.getRRCed,        'getMany': rx.getManyRRCed,        'fs':16384}
}

tap = 'Decimated'
#     ^^^^^^^^^^^
#    Pick between Decimated, CoarseSynced, RRCed, or Data

f=taps[tap]['get']
d=taps[tap]['getMany']()
fs=taps[tap]['fs']

f_plot = IQFreqPlot(f(), fs)
f_dg = DmaGen(f_plot.add_frame, f, 0.3)

iq_plot = IQTimePlot(d, fs, w=800)
iq_dg = DmaGen(iq_plot.add_data, f, 0.05)

c_plot = IQConstellationPlot(d, plotrange=(0, len(d)-1), fade=True)
c_dg = DmaGen(c_plot.add_data, f, 0.05)

tab1 = ipw.Tab([ipw.VBox([iq_plot.get_widget(), ipw.HBox(GraphControls(iq_dg))]),
                ipw.VBox([f_plot.get_widget(), ipw.HBox(GraphControls(f_dg))]),
                ipw.VBox([c_plot.get_widget(), ipw.HBox(GraphControls(c_dg))]),])
tab1.set_title(0, 'Time domain')
tab1.set_title(1, 'Frequency domain')
tab1.set_title(2, 'Constellation')
tab1

# Output data

Finally we can plot a constellation of the final, synchronised signal, showing the 4 constellations expected of QPSK. The transparency of each sample represents its "age", as there in no explicit time axis for constellation plots. On our test system, ~20 frames per second is achievable.

In [ ]:
# Pick between getDecimated, getCoarseSynced, getRRCed, or getData
d=rx.getManyData()
fs=500

iq_plot = IQTimePlot(d, fs, w=800)
iq_dg = DmaGen(iq_plot.add_data, rx.getData, 0.05)

c_plot = IQConstellationPlot(d, plotrange=(0, len(d)-1), fade=True)
c_dg = DmaGen(c_plot.add_data, rx.getData, 0.05)

tab1 = ipw.Tab([ipw.VBox([c_plot.get_widget(), ipw.HBox(GraphControls(c_dg))]),
                ipw.VBox([iq_plot.get_widget(), ipw.HBox(GraphControls(iq_dg))])])
tab1.set_title(0, 'Constellation')
tab1.set_title(1, 'Time domain')
tab1

## Scratch pad

Everything below this point is not part of the main demo --- just an environment for testing and having a play around.

In [ ]:
from IPython.display import Image
Image("img/combined.png")

## Profiling of plotting code

Note: need to pip install line_profiler before this will work.

Very useful approach for spotting any server-side bottleneck in our live plots.
Cell below is marked as "Raw", so we can "Run all cells" without breaking anything.

...or alternatively, just use a simple timer with no line-by-line feedback